In [ ]:
%load_ext autoreload
%autoreload 2
%aimport

In [ ]:
%matplotlib inline 

In [ ]:
import matplotlib.pyplot as plt
import torch

from bliss.inference import SDSSFrame
from bliss.datasets import sdss
from bliss import sleep
from bliss.inference import reconstruct_scene_at_coordinates
from bliss.reporting import plot_image
from case_studies.sdss_galaxies.plots import load_models
from bliss.datasets.sdss import convert_flux_to_mag


from astropy.table import Table
from astropy.io import fits

import ipywidgets as widgets
from ipywidgets import interact, interactive, fixed, interact_manual

import plotly.express as px
import plotly.graph_objects as go

In [ ]:
# check GPU is configured correctly
device = torch.device('cuda:0')
!echo $CUDA_VISIBLE_DEVICES

# Load data

In [ ]:
# load sdss data
sdss_dir = '/home/imendoza/bliss/data/sdss/'
pixel_scale = 0.393
coadd_file = "/home/imendoza/bliss/data/coadd_catalog_94_1_12.fits"
frame = SDSSFrame(sdss_dir, pixel_scale, coadd_file)

In [ ]:
# load models
from hydra import compose, initialize
from hydra.utils import instantiate
from bliss.encoder import Encoder

with initialize(config_path="../config"):
    cfg = compose("config", overrides=[])
    
    
enc, dec = load_models(cfg, device)
bp = enc.border_padding
torch.cuda.empty_cache()

In [ ]:
# get catalog 
h, w = bp, bp
h_end = ((frame.image.shape[2] - 2 * bp) // 4) * 4 + bp
w_end = ((frame.image.shape[3] - 2 * bp) // 4) * 4 + bp
coadd_params = frame.get_catalog((h, h_end), (w, w_end))

In [ ]:
# frame PHOTO catalog
frame_cat = Table.read('../../../data/sdss/94/1/12/photoObj-000094-1-0012.fits')

In [ ]:
print("# objects detected by photo on frame: ", len(frame_cat))
print("# objects detected by photo on coadd: ", coadd_params.n_sources.sum().item())

# Get Locations

In [ ]:
#inference
with torch.no_grad():
    _, tile_est = reconstruct_scene_at_coordinates(
        enc,
        dec,
        frame.image,
        frame.background,
        h_range=(h, h_end),
        w_range=(w, w_end),
        slen=300,
        device=device,
    )
map_recon = tile_est.to_full_params()
map_recon["fluxes"] = (
    map_recon["galaxy_bools"] * map_recon["galaxy_fluxes"]
    + map_recon["star_bools"] * map_recon["fluxes"]
)
map_recon["mags"] = sdss.convert_flux_to_mag(map_recon["fluxes"])
torch.cuda.empty_cache()

In [ ]:
# prepare inference locs
plocs = map_recon.plocs.cpu().numpy().squeeze() + bp
coords = frame.wcs.all_pix2world(np.hstack([plocs[:, 1, None], plocs[:, 0, None]]), 0)
galaxy_bool = map_recon['galaxy_bools'].numpy().astype(bool).squeeze()
galaxy_prob = map_recon['galaxy_probs'].numpy().squeeze()
mags = map_recon['mags'].squeeze().numpy()


In [ ]:
# coadd params
mag_cut = 24
_coadd_params = coadd_params.apply_mag_bin(14, mag_cut)
coplocs = _coadd_params.plocs.squeeze().numpy() + bp
cogbool = _coadd_params['galaxy_bools'].squeeze().numpy()

cora = _coadd_params['ra'].squeeze().numpy()
codec = _coadd_params['dec'].squeeze().numpy()
comags = _coadd_params['mags'].squeeze().numpy()

# Plotly Frame

In [ ]:
# extract misclassified objects between 19.5 and 20.5
from bliss.reporting import match_by_locs

coindx, indx, dkeep, _ = match_by_locs(torch.from_numpy(coplocs), torch.from_numpy(plocs))
nindx = np.array(list(set(range(len(plocs))).difference(set(indx))))
assert len(indx) + len(nindx) == len(plocs)

mags_keep = comags[coindx][dkeep] < 19
gal_misclass = ~np.equal(galaxy_bool[indx][dkeep][mags_keep], cogbool[coindx][dkeep][mags_keep]) & cogbool[coindx][dkeep][mags_keep]
plocs_misclassified = plocs[indx][dkeep][mags_keep][gal_misclass]
plocs_unmatched = np.vstack([plocs[nindx], plocs[indx][~dkeep]])


In [ ]:
image = frame.image.squeeze().numpy()

fig = px.imshow(image, width=800, height=550, zmin=800, zmax=1150)


# create scatter objects
scatter_coadd_galaxies = go.Scatter(name='galaxy coadd', x=coplocs[cogbool][:, 1], y=coplocs[cogbool][:, 0], 
                                  mode='markers', marker=dict(color='red', size=8, symbol='cross'), 
                                 hovertext=[f'mag:{x:.2f}; (ra, dec):({y:.4f}, {z:.4f})' 
                                            for (x, y,z) in zip(comags[cogbool], 
                                                             cora[cogbool],
                                                             codec[cogbool])]
                                 )
scatter_coadd_star = go.Scatter(name='star coadd', x=coplocs[~cogbool][:, 1], y=coplocs[~cogbool][:, 0], 
                                  mode='markers', marker=dict(color='blue', size=8, symbol='cross'),
                                 hovertext=[f'mag:{x:.2f}; (ra, dec):({y:.4f}, {z:.4f})' 
                                            for (x, y,z) in zip(comags[~cogbool], 
                                                             cora[~cogbool],
                                                             codec[~cogbool])]
                                 )
scatter_bliss_galaxies = go.Scatter(name='galaxy pred', x=plocs[galaxy_bool][:, 1], y=plocs[galaxy_bool][:, 0], 
                                  mode='markers', marker=dict(color='hotpink', size=11, symbol='x'),
                                 hovertext=[f'mag:{a:.2f}; prob_galaxy:{x:.2f}; (ra, dec):({y:.4f}, {z:.4f})' 
                                            for (a, x, y,z) in zip(mags[galaxy_bool], 
                                                               galaxy_prob[galaxy_bool], 
                                                               coords[galaxy_bool][:, 0], 
                                                               coords[galaxy_bool][:, 1])]
                                 )
scatter_bliss_star = go.Scatter(name='star pred', x=plocs[~galaxy_bool][:, 1], y=plocs[~galaxy_bool][:, 0], 
                                mode='markers', marker=dict(color='cyan', size=11, symbol='x'),\
                                 hovertext=[f'mag:{a:.2f}; prob_galaxy:{x:.2f}; (ra, dec):({y:.4f},{z:.4f})' 
                                            for (a, x,y,z) in zip(mags[~galaxy_bool], 
                                                               galaxy_prob[~galaxy_bool], 
                                                               coords[~galaxy_bool][:, 0], 
                                                               coords[~galaxy_bool][:, 1])]
                                 )

scatter_unmatched = go.Scatter(name='unmatched', x=plocs_unmatched[:, 1], y=plocs_unmatched[:, 0], 
                                mode='markers', marker=dict(color='magenta', size=10, symbol='circle'),
                                 )
scatter_misclassified = go.Scatter(name='misclassified', x=plocs_misclassified[:, 1], y=plocs_misclassified[:, 0], 
                                mode='markers', marker=dict(color='yellow', size=10, symbol='x'),
                                 )

# add traces to figure
fig.add_trace(scatter_coadd_galaxies)
fig.add_trace(scatter_coadd_star)
fig.add_trace(scatter_bliss_galaxies)
fig.add_trace(scatter_bliss_star)
fig.add_trace(scatter_unmatched)
fig.add_trace(scatter_misclassified)


fig.update_layout(legend=dict(orientation="h", y=1.05)) # adjust legend


fig.show()